In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from openai import AzureOpenAI
from pprint import pprint
from utils.prompts.prompt import *
from utils.prompts.Schema_Prompt_v1 import *
from utils.postgres_functions import *

In [2]:
load_dotenv()

True

In [3]:
def calculate_confidence_score(logprobs: list) -> float:
    probs = []
    # iterate through the token predictions and get the linear probs [0;1]
    for logprob in logprobs:
        probs.append(np.round(np.exp(logprob)*100, 2))
    # calculate the confidence
    return round(np.mean(probs), 2)

In [4]:
client = AzureOpenAI(
    api_key = str(os.environ['azure_api_key']),
    api_version = str(os.environ['azure_api_version']),
    azure_endpoint = str(os.environ['azure_api_endpoint'])
)

In [118]:
# QUERY = "Forgalomban van a novocetrin?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok filmtablettás gyógyszerként kiírni?"
QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok kiírni?"
# QUERY = "Mennyibe kerül a syntroxine?"
# QUERY = "Milyen gyógyszert lenne érdemes felírni láz esetén?"
# QUERY = "Milyen kiszerelésben kapható az egilok tabletta?"
# QUERY = "Ki forgalmazza a perdox oldatot?"
# QUERY = "Mivel lehet kiváltani a Matrifent?"
# QUERY = "Ki írhatja fel a letrozol pharcet?"
# QUERY = "Milyen betegségek esetén alkalmazható a Brusimp?"
# QUERY = "Mi a calcium fluoratum ATC kódjának értelmezése?"

In [119]:
sql_gen_messages = [
                {"role": "system", "content" : sql_system_message.format(system_prompt=SQL_SYSTEM_PROMPT, 
                                                                    table_info=SCHEMA_PROMPT)},
                {"role": "user", "content": question_message.format(question=QUERY)}
                ]

In [120]:
full_response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=sql_gen_messages,
    temperature=0.0,
    max_tokens=1024,
    logprobs=True
)
pprint(full_response.choices[0].message.content)

"SELECT OEP_NEV, HATOANYAG FROM GYOGYSZ WHERE HATOANYAG = 'ciszplatin' LIMIT 10"


In [113]:
pprint(full_response)

ChatCompletion(id='chatcmpl-9FnVdZP1Vk4O14znh6wl51gJO6Alv', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='SELECT', bytes=[83, 69, 76, 69, 67, 84], logprob=-0.0063932342, top_logprobs=[]), ChatCompletionTokenLogprob(token=' G', bytes=[32, 71], logprob=-0.5799946, top_logprobs=[]), ChatCompletionTokenLogprob(token='Y', bytes=[89], logprob=-0.001165461, top_logprobs=[]), ChatCompletionTokenLogprob(token='OG', bytes=[79, 71], logprob=-0.00022487417, top_logprobs=[]), ChatCompletionTokenLogprob(token='YS', bytes=[89, 83], logprob=-9.0883464e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token='Z', bytes=[90], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token='.O', bytes=[46, 79], logprob=-0.43099958, top_logprobs=[]), ChatCompletionTokenLogprob(token='EP', bytes=[69, 80], logprob=-4.00813e-06, top_logprobs=[]), ChatCompletionTokenLogprob(token='_NE', bytes=[95, 78, 69], logprob=-0.00024989594, top_logprobs=

In [114]:
logprobs = [x.logprob for x in full_response.choices[0].logprobs.content]
calculate_confidence_score(logprobs)

95.44

In [121]:
db_connection = connect_to_db(str(os.environ['db_host']), str(os.environ['db_name']), str(os.environ['db_user']), str(os.environ['db_password']))
print(read_db(full_response.choices[0].message.content,db_connection))
close_connection(db_connection)

Connection established
                                                   0           1
0  CISPLATIN SANDOZ 0,5 MG/ML KONCENTRÁTUM OLDATO...  ciszplatin
1  CISPLATIN SANDOZ 0,5 MG/ML KONCENTRÁTUM OLDATO...  ciszplatin
2  CISPLATIN SANDOZ 0,5 MG/ML KONCENTRÁTUM OLDATO...  ciszplatin
3  CISPLATIN ACCORD 1 MG/ML KONCENTRÁTUM OLDATOS ...  ciszplatin
4  CISPLATIN ACCORD 1 MG/ML KONCENTRÁTUM OLDATOS ...  ciszplatin
5  CISPLATIN ACCORD 1 MG/ML KONCENTRÁTUM OLDATOS ...  ciszplatin
6  CISPLATIN ACCORD 1 MG/ML KONCENTRÁTUM OLDATOS ...  ciszplatin


In [129]:
db_connection = connect_to_db(str(os.environ['db_host']), str(os.environ['db_name']), str(os.environ['db_user']), str(os.environ['db_password']))
print(read_db("SELECT * from ORVOSOK",db_connection))
close_connection(db_connection)

Connection established
Empty DataFrame
Columns: []
Index: []
